In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [287]:
# Pkg.add("BenchmarkTools");

   Resolving package versions...
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d/Manifest.toml`


In [288]:
using Revise
using ModelingToolkit, LinearAlgebra
using GLMakie
using ControlSystems
using DifferentialEquations
using StaticArrays
using BenchmarkTools

include("utilities.jl")
include("types.jl")
include("linearize.jl")

linearize_system (generic function with 1 method)

## Forward-diff demo

In [365]:
let

x(r, ω, t) = 1 * cos(1*t)
x(var) = x(var[1],var[2], var[3])

ẋ_fdiff =  ForwardDiff.gradient(x, [1, 1, 0.5])

# compare with analytical derivative
ẋ(r, ω, t) = -r * sin(ω*t) * ω

ẋ_analytical = ẋ(1, 1, 0.5)

# show results 
println("ẋ Analytical: $(ẋ_analytical)")
println("ẋ Forward-diff: $(ẋ_fdiff)")

end

ẋ Analytical: -0.479425538604203
ẋ Forward-diff: [-0.0, -0.0, -0.479425538604203]


In [394]:
let 
include("trajectory_generation.jl")

t = 1.2 # seconds
(x_req, y_req, θ_req, ẋ_req, ẏ_req, θ̇_req) = generate_circle_trajectory(circle_trajec_params, t)

end

6-element Vector{Float64}:
 2.309016994374948
 2.9510565162951536
 0.10158590543965393
 0.0
 0.0
 0.0

## Symbolics.jl function generation demo (theta_dot for 2d quadcopter differential flataness)

In [489]:
# let 
    
function get_θ̇()
    @variables t 
    D = Differential(t)
    
    @parameters ω m r
    
    @constants g=-9.81

    y = r*cos(ω*t)
    z = r*sin(ω*t)

    ẏ = D(y)
    ż = D(z)

    ÿ = D(ẏ)
    z̈ = D(ż)
    
    θ = expand_derivatives(atan(-m*ÿ, m*(z̈ -g)))
        
    θ̇ = expand_derivatives(D(θ))

     # build functions for θ and θ̇
    f_θ_expr = build_function(θ, [r,ω,m,g,t])
    f_θ̇_expr = build_function(θ̇, [r,ω,m,g,t])
        
    Base.remove_linenums!(f_θ_expr)
    Base.remove_linenums!(f_θ̇_expr)
        
     f_θ = eval(f_θ_expr)
     f_θ̇ = eval(f_θ̇_expr)

#     # # to build C function
#     # build_function(ÿ, [y,ω], target=Symbolics.CTarget())
    
     return f_θ, f_θ̇
end

let 

(f_θ, f_θ̇) = get_θ̇()

params = (; ω=0.2*π, m=1.0 ,g=-9.81, r=1.0)
    
θ::Float64= f_θ(SA_F64[params.r, params.ω, params.m, params.g, 0.2])
θ̇::Float64 = f_θ̇(SA_F64[params.r, params.ω, params.m, params.g, 0.2])

println("θ: ", θ);
println("θ̇: ", θ̇);
        
end

θ: 0.04010658647214026
θ̇: -0.002169918550128466


### Verification of f_θ 

In [310]:
let

params = (; ω=0.2*π, m=1.0 ,g=-9.81, r=1.0)

y(r,ω,t) = r*cos(ω*t)
z(r,ω,t) = r*sin(ω*t)
    
ÿ(r,ω,t) = -y(r,ω,t) * (ω^2)
z̈(r,ω,t) = -z(r,ω,t) * (ω^2)

# Analytical output 
t::Float64 = 0.2
θ_analytical::Float64 =  atan(-params.m * ÿ(params.r, params.ω, t) , params.m * (z̈(params.r, params.ω, t)  + 9.81))

# Output from symbolics.jl
(f_θ, f_θ̇) = get_θ̇()
θ_symbolics::Float64= f_θ(SA_F64[params.r, params.ω, params.m, params.g, 0.2])

# show results 
println("θ_analytical: ", θ_analytical)
println("θ_symbolics: ", θ_symbolics)

end

θ_analytical: 0.04010658647214026
θ_symbolics: 0.04010658647214026


## Forwardfiff for (theta_dot for 2d quadcopter differential flataness)

In [528]:
let

using ForwardDiff

params = (; ω=0.2*π, m=1.0 ,g=-9.81, r=1.0)
    
y(r::Real,ω::Real,t::Real) = (r*cos(ω*t)) 
z(r::Real,ω::Real,t::Real) = (r*sin(ω*t)) 
    
ÿ(r::Real,ω::Real,t::Real) = -y(r,ω,t) * (ω^2)
z̈(r::Real,ω::Real,t::Real) = -z(r,ω,t) * (ω^2)
    
function θ(t::Real, params)
        m::Float64 = params.m
        r::Float64 = params.r
        ω::Float64 = params.ω
        g::Float64 = params.g
        
        return atan(-m * ÿ(r,ω,t), m*(z̈(r,ω,t) -g))
end   

θ(var::Real) = θ(var, params)

# result from forwarddiff.jl
θ̇_fdiff =  ForwardDiff.derivative(θ, 0.2)

# result from symbolics.jl
(f_θ, f_θ̇) = get_θ̇()
θ_symbolics::Float64=  f_θ̇(SA_F64[params.r, params.ω, params.m, params.g, 0.2])

# show results 
println("θ_forwarddiff: ", θ̇_fdiff)
println("θ_symbolics: ", θ_symbolics)
    
end

θ_forwarddiff: -0.002169918550128466
θ_symbolics: -0.002169918550128466


## Benchmarking 

### Symbolics.jl generated

In [529]:
let 

(f_θ, f_θ̇) = get_θ̇()

params = (; ω=0.2*π, m=1.0 ,g=-9.81, r=1.0)
    
@timev f_θ̇(SA_F64[params.r, params.ω, params.m, params.g, 0.2])
        
end

  0.023545 seconds (43.67 k allocations: 2.766 MiB, 99.91% compilation time)
elapsed time (ns):  23544527
gc time (ns):       0
bytes allocated:    2900702
pool allocs:        43648
non-pool GC allocs: 18
minor collections:  0
full collections:   0


-0.002169918550128466

### Forwarddiff.jl

In [535]:
let 

params = (; ω=0.2*π, m=1.0 ,g=-9.81, r=1.0)
    
y(r::Real,ω::Real,t::Real) = (r*cos(ω*t)) 
z(r::Real,ω::Real,t::Real) = (r*sin(ω*t)) 
    
ÿ(r::Real,ω::Real,t::Real) = -y(r,ω,t) * (ω^2)
z̈(r::Real,ω::Real,t::Real) = -z(r,ω,t) * (ω^2)
    
function θ(t::Real, params)
        m::Float64 = params.m
        r::Float64 = params.r
        ω::Float64 = params.ω
        g::Float64 = params.g
        
        return atan(-m * ÿ(r,ω,t), m*(z̈(r,ω,t) -g))
end   

θ(var::Real) = θ(var, params)

@timev ForwardDiff.derivative(θ, 0.2);  
    
end

  0.049163 seconds (39.10 k allocations: 2.599 MiB, 99.80% compilation time)
elapsed time (ns):  49162844
gc time (ns):       0
bytes allocated:    2724963
pool allocs:        39090
non-pool GC allocs: 6
minor collections:  0
full collections:   0


-0.002169918550128466